In [1]:
import strawberry
from pathlib import Path
from typing import Optional, List, Any
from datetime import datetime

# sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.engine.base import Engine
from sqlalchemy.orm import Session, sessionmaker, Query, joinedload, load_only

# highlighting sql statements
import sqlparse
from pygments import highlight
from pygments.formatters.terminal import TerminalFormatter
from pygments.lexers import SqlLexer

# utils
from pprint import pprint

# rootski
import rootski.services.database.models as orm
from rootski import schemas  # pydantic models

In [2]:
############################
# --- Helper Functions --- #
############################

SQLITE_DB_FPATH = Path(
    "/Users/eric/repos/extra/rootski/rootski/rootski_api/migrations/initial_data/rootski.sqlite"
)
SQLITE_URI = "sqlite:///" + str(SQLITE_DB_FPATH)


def make_engine(**engine_kwargs) -> Engine:
    return create_engine(SQLITE_URI, **engine_kwargs)


def get_session(engine: Engine) -> Session:
    return sessionmaker(bind=engine)()


def get_raw_sql(query: Query, engine: Engine) -> str:
    """Convert a SQLAlchemy query object to a nicely formatted, colorized SQL query string."""
    compiled_sql = query.statement.compile(engine, compile_kwargs={"literal_binds": True})
    return str(compiled_sql)


def prettify_sql(sql: str) -> str:
    """Format and colorize a SQL string. Requires pygments and sqlparse to be installed."""
    parsed = sqlparse.format(sql, reindent=True, keyword_case="upper")
    return highlight(parsed, SqlLexer(), TerminalFormatter())


def pprint_query(query: Query, engine: Engine):
    raw_sql: str = get_raw_sql(query=query, engine=engine)
    pretty_sql: str = prettify_sql(sql=raw_sql)
    print(pretty_sql)

In [3]:
############################
# --- Strawberry Types --- #
############################

# these are almost exactly the ORM models, but many fields are set as "Optional"
from __future__ import annotations

# is there a better way to do this SQLAlchemy model attribute check?
def get_or_none(model, field: str) -> Optional[Any]:
    """Return the attribute on the sqlalchemy model or None if it has not been loaded."""
    if field in model.__dict__.keys() and hasattr(model, field):
        return getattr(model, field)
    return None


@strawberry.type
class Word:
    id: strawberry.ID
    word: Optional[str] = None
    accent: Optional[str] = None
    frequency: Optional[str] = None
    breakdowns: Optional[List[Breakdown]] = None

    # pydantic writes this "from_orm" method for us; it also takes care of json serialization/deserialization,
    # can we use strawberry's experimental Pydantic extension to save us from having to write this?
    # Or is there another way?
    @classmethod
    def from_orm(cls, word: orm.Word) -> Word:
        return cls(
            id=get_or_none(word, "id"),
            word=get_or_none(word, "word"),
            accent=get_or_none(word, "accent"),
            frequency=get_or_none(word, "frequency"),
            breakdowns=[Breakdown.from_orm(b) for b in word.breakdowns]
            if get_or_none(word, "breakdowns")
            else [],
        )


@strawberry.type
class Breakdown:
    word_id: Optional[strawberry.ID]
    breakdown_id: Optional[strawberry.ID]
    word: Optional[str]
    submitted_by_user_email: Optional[str]
    verified_by_user_email: Optional[str]
    is_verified: Optional[bool]
    is_inference: Optional[bool]
    date_submitted: Optional[str]
    date_verified: Optional[datetime]
    breakdown_items: Optional[List[BreakdownItem]]  # one to many
    word_: Optional[Word]
    # verified_by_user: Optional[User] # TODO: implement the User type
    # submitted_by_user: Optional[User]

    @classmethod
    def from_orm(cls, breakdown: orm.Breakdown) -> Breakdown:
        return cls(
            word_id=get_or_none(breakdown, "word_id"),
            breakdown_id=get_or_none(breakdown, "breakdown_id"),
            word=get_or_none(breakdown, "word"),
            submitted_by_user_email=get_or_none(breakdown, "submitted_by_user_email"),
            verified_by_user_email=get_or_none(breakdown, "verified_by_user_email"),
            is_verified=get_or_none(breakdown, "is_verified"),
            is_inference=get_or_none(breakdown, "is_inference"),
            date_submitted=get_or_none(breakdown, "date_submitted"),
            date_verified=get_or_none(breakdown, "date_verified"),
            breakdown_items=[BreakdownItem.from_orm(b) for b in breakdown.breakdown_items]
            if get_or_none(breakdown, "breakdown_items")
            else [],
            word_=get_or_none(breakdown, "word_"),
        )


@strawberry.type
class BreakdownItem:
    id: strawberry.ID
    morpheme_id: Optional[strawberry.ID]
    breakdown_id: Optional[strawberry.ID]
    morpheme: Optional[str]
    type: Optional[str]  # TODO: make this an enum
    position: Optional[int]
    # morpheme_: Optional[Morpheme]
    breakdown: Optional[Breakdown]  # many to one

    @classmethod
    def from_orm(cls, b_item: orm.BreakdownItem) -> BreakdownItem:
        return cls(
            id=get_or_none(b_item, "id"),
            morpheme_id=get_or_none(b_item, "morpheme_id"),
            breakdown_id=get_or_none(b_item, "breakdown_id"),
            morpheme=get_or_none(b_item, "morpheme"),
            type=get_or_none(b_item, "type"),
            position=get_or_none(b_item, "position"),
            breakdown=get_or_none(b_item, "breakdown"),
        )

In [4]:
# try not to run this cell often, we don't have logic in place to close the session
engine: Engine = make_engine(echo=False)
session: Session = get_session(engine)

In [5]:
"""
query {
    words(offset:5, limit:5, contains:"вы") {
       id
       word
       breakdowns {
           breakdown_id
           is_verified
           breakdown_items {
              morpheme_id
              morpheme
           }
       }
    }
}
"""

# args
contains = "вы"
offset = 5
limit = 2

# sqlalchemy query
query: Query = (
    session.query(orm.Word)
    .filter(orm.Word.word.ilike(f"%{contains}%"))  # ILIKE is case insensitive LIKE
    .offset(offset)  # set offset and limit (for pagination)
    .limit(limit)
    .options(
        load_only(orm.Word.id, orm.Word.word),  # only load certain word fields
        joinedload(orm.Word.breakdowns).options(  # eager load the breakdowns for the word (one to many)
            load_only(
                orm.Breakdown.breakdown_id, orm.Breakdown.is_verified
            ),  # only load certain breakdown fields
            joinedload(
                orm.Breakdown.breakdown_items
            ).options(  # eager load the breakdown items for the breakdown (one to many)
                load_only(
                    orm.BreakdownItem.morpheme_id, orm.BreakdownItem.morpheme
                ),  # only load certain breakdown item fields
            ),
        ),
    )
)

pprint_query(query, engine)

SELECT anon_1.id,
       anon_1.word,
       breakdowns_1.id AS id_1,
       breakdowns_1.morpheme_id,
       breakdowns_1.morpheme,
       word_to_breakdowns_1.breakdown_id,
       word_to_breakdowns_1.is_verified
FROM
  (SELECT words.id AS id,
          words.word AS word
   FROM words
   WHERE lower(words.word) LIKE lower('%вы%')
   LIMIT 2
   OFFSET 5) AS anon_1
LEFT OUTER JOIN word_to_breakdowns AS word_to_breakdowns_1 ON anon_1.id = word_to_breakdowns_1.word_id
LEFT OUTER JOIN breakdowns AS breakdowns_1 ON word_to_breakdowns_1.breakdown_id = breakdowns_1.breakdown_id



In [6]:
# Is there a way we can turn these into JSON?
orm_words: orm.Word = query.all()
words = [Word.from_orm(word) for word in orm_words]

In [7]:
words

[Word(id=438, word='выходить', accent=None, frequency=None, breakdowns=[Breakdown(word_id=None, breakdown_id=424, word=None, submitted_by_user_email=None, verified_by_user_email=None, is_verified=False, is_inference=None, date_submitted=None, date_verified=None, breakdown_items=[BreakdownItem(id=902, morpheme_id=218, breakdown_id=None, morpheme='вы', type=None, position=None, breakdown=None), BreakdownItem(id=903, morpheme_id=1577, breakdown_id=None, morpheme='ход', type=None, position=None, breakdown=None), BreakdownItem(id=904, morpheme_id=2139, breakdown_id=None, morpheme='ить', type=None, position=None, breakdown=None)], word_=None)]),
 Word(id=485, word='правый', accent=None, frequency=None, breakdowns=[Breakdown(word_id=None, breakdown_id=471, word=None, submitted_by_user_email=None, verified_by_user_email=None, is_verified=False, is_inference=None, date_submitted=None, date_verified=None, breakdown_items=[BreakdownItem(id=1026, morpheme_id=None, breakdown_id=None, morpheme='ый',

In [8]:
from fastapi import FastAPI, Depends, Request
from fastapi.testclient import TestClient

from strawberry.asgi import GraphQL
from strawberry import Schema
from strawberry.types import Info

from pydantic import BaseModel

# this is copy/pasted from the the currently open PR for the FastAPI integration, hahaha
from rootski.main.endpoints.strawberry.fastapi import GraphQLRouter


class GraphQLContext(BaseModel):
    current_user: schemas.User
    db: Session

    class Config:
        arbitrary_types_allowed = True  # prevent pydantic from getting mad about us using "Session" as a type


################################
# --- FastAPI Dependencies --- #
################################


def get_authorized_user_email_or_anon(request: Request, logged_in: bool = False) -> schemas.User:
    # we *could* get the user email from the JWT found in the headers on the request object in this function
    if logged_in:
        return schemas.User(email="banana-man@rootski.io", is_admin=True)
    return schemas.User(email="anon@rootski.io", is_admin=False)


def get_db(request: Request) -> Session:
    engine: Engine = make_engine()
    session: Session = get_session(engine=engine)
    return session


def get_graphql_context(
    db: Session = Depends(get_db), current_user: schemas.User = Depends(get_authorized_user_email_or_anon)
):
    return GraphQLContext(db=db, current_user=current_user)


#########################
# --- GraphQL Setup --- #
#########################


def resolve_words(info: Info, offset: int, limit: int, contains: str) -> List[Word]:
    ctx: GraphQLContext = info.context
    session = ctx.db

    # ideally, we would create this SQLAlchemy query by recursively traversing the GraphQL
    # Abstract Syntax Tree (AST), raising an error if the number of joins AKA the depth of
    # the query is greater than some value (maybe 3?)
    query: Query = (
        session.query(orm.Word)
        .filter(orm.Word.word.ilike(f"%{contains}%"))  # ILIKE is case insensitive LIKE
        .offset(offset)  # set offset and limit (for pagination)
        .limit(limit)
        .options(
            load_only(orm.Word.id, orm.Word.word),  # only load certain word fields
            joinedload(orm.Word.breakdowns).options(  # eager load the breakdowns for the word (one to many)
                load_only(
                    orm.Breakdown.breakdown_id, orm.Breakdown.is_verified
                ),  # only load certain breakdown fields
                joinedload(
                    orm.Breakdown.breakdown_items
                ).options(  # eager load the breakdown items for the breakdown (one to many)
                    load_only(
                        orm.BreakdownItem.morpheme_id, orm.BreakdownItem.morpheme
                    ),  # only load certain breakdown item fields
                ),
            ),
        )
    )

    orm_words = query.all()

    return [Word.from_orm(word) for word in orm_words]


@strawberry.type
class RootQuery:
    words: List[Word] = strawberry.field(resolver=resolve_words)


# to get POST /graphql and GET /graphql to enter into this route,
# I hardcoded the string "/graphql" into my copy of GraphQLRouter...
# I wasn't sure what the appropriate way to do that was
graphql_router = GraphQLRouter(
    schema=Schema(query=RootQuery), graphiql=True, context_getter=get_graphql_context
)


#########################
# --- FastAPI Setup --- #
#########################

app = FastAPI()
app.include_router(graphql_router)

In [9]:
client = TestClient(app)

query = """
query {
    words(offset:5, limit:2, contains:"вы") {
       id
       word
       breakdowns {
           breakdownId
           isVerified
           breakdownItems {
              morphemeId
              morpheme
           }
       }
    }
}
"""
response = client.post("/graphql", json={"query": query}, headers={"Content-Type": "application/json"})

In [10]:
import json

response = response.json()
print(json.dumps(response, indent=4))

{
    "data": {
        "words": [
            {
                "id": "438",
                "word": "\u0432\u044b\u0445\u043e\u0434\u0438\u0442\u044c",
                "breakdowns": [
                    {
                        "breakdownId": "424",
                        "isVerified": false,
                        "breakdownItems": [
                            {
                                "morphemeId": "218",
                                "morpheme": "\u0432\u044b"
                            },
                            {
                                "morphemeId": "1577",
                                "morpheme": "\u0445\u043e\u0434"
                            },
                            {
                                "morphemeId": "2139",
                                "morpheme": "\u0438\u0442\u044c"
                            }
                        ]
                    }
                ]
            },
            {
                "id": "485",
 